# Pandas - Korelační koeficienty krok za krokem

## Úvod do korelačních koeficientů

Tento notebook vysvětluje různé typy korelačních koeficientů od úplného základu pro ty, kteří se s korelacemi setkávají poprvé.

### Co je korelace?

**Korelace** měří, jak moc spolu dvě věci "jdou ruku v ruce". 

Představte si:
- Když je teplo, lidé více pijí zmrzlinu → **pozitivní korelace**
- Když je teplo, méně lidí nosí kabáty → **negativní korelace**  
- Počet bot, které máte doma, a vaše IQ → **žádná korelace**

### Základní fakta o korelaci
- Korelace je vždy mezi **-1 a +1**
- **+1** = perfektní pozitivní vztah (jedno roste, druhé také)
- **-1** = perfektní negativní vztah (jedno roste, druhé klesá)
- **0** = žádný vztah

### Proč existuje více typů korelačních koeficientů?

Představte si tři různé situace:

#### Situace 1: Přímý vztah
```
Teplota:  10°C → 15°C → 20°C → 25°C
Zmrzlina:  2ks →  4ks →  6ks →  8ks
```
Každých +5°C = +2 zmrzliny. **Přímý lineární vztah.**

#### Situace 2: Vztah, ale ne přímý
```
Rychlost auta: 30 km/h → 60 km/h → 90 km/h → 120 km/h
Spotřeba:       5 l/h  →  8 l/h  → 15 l/h →  25 l/h
```
Rychlejší auto = větší spotřeba, ale ne přímo úměrně. **Nelineární vztah.**

#### Situace 3: Pořadí je důležité
```
Závod - umístění: 1. → 2. → 3. → 4.
Počet bodů:      100 → 85 → 70 → 50
```
Lepší umístění = více bodů, ale rozdíly mezi místy nejsou stejné.

**Proto potřebujeme různé korelační koeficienty pro různé situace!**

## Přehled korelačních koeficientů

| Koeficient | Kdy použít | Co měří |
|------------|------------|---------|
| **Pearson** | Přímý lineární vztah | Sílu lineární závislosti |
| **Spearman** | Vztah podle pořadí | Monotónní závislost |
| **Kendall** | Malé vzorky, odlehlé hodnoty | Pořadovou závislost |

### Pandas metody a parametry

| Metoda | Popis | Příklad |
|--------|-------|---------|
| `series.corr(other, method='pearson')` | Korelace mezi dvěma Series | `df['a'].corr(df['b'])` |
| `df.corr(method='pearson')` | Korelační matice DataFrame | `df.corr()` |
| `series.rank()` | Převod na pořadí (ranky) | `df['col'].rank()` |
| `series.diff()` | Rozdíly mezi hodnotami | `df['col'].diff()` |
| `series.is_monotonic_increasing` | Test monotónního růstu | `df['col'].is_monotonic_increasing` |
| `series.dropna()` | Odstranění NaN hodnot | `df['col'].dropna()` |
| `series.tolist()` | Převod na Python list | `df['col'].tolist()` |
| `series.round()` | Zaokrouhlení hodnot | `df['col'].round(3)` |
| `series.abs()` | Absolutní hodnota | `df['col'].abs()` |

### Parametry pro korelační metody

| Parametr | Hodnota | Koeficient |
|----------|---------|------------|
| `method='pearson'` | Výchozí | Pearsonův |
| `method='spearman'` | Pro pořadí | Spearmanův |
| `method='kendall'` | Pro malé vzorky | Kendallův |

## Pearsonův korelační koeficient

**Kdy použít:** Když očekáváte přímý lineární vztah (jedno roste, druhé roste stejným tempem).

### Jak Pearsonův koeficient funguje:

- **Měří lineární vztah** mezi dvěma proměnnými
- **Počítá se** z kovariance vydělené součinem směrodatných odchylek
- **Citlivý na odlehlé hodnoty** - jedna extrémní hodnota může změnit výsledek
- **Předpokladá** normální rozdělení dat
- **Nejlepší** pro spojitá číselná data s lineárním vztahem

### Krok za krokem s jednoduchým příkladem

In [ ]:
import pandas as pd
import numpy as np

# Velmi jednoduchá data - teplota vs prodej zmrzliny
jednoducha_data = pd.DataFrame({
    'teplota': [10, 15, 20, 25, 30],
    'zmrzlina': [2, 4, 6, 8, 10]
})

print("Naše data:")
print(jednoducha_data)
print()

# Pearsonova korelace
pearson = jednoducha_data['teplota'].corr(jednoducha_data['zmrzlina'], method='pearson')
print(f"Pearsonova korelace: {pearson:.3f}")

### Proč je výsledek 1.000?

In [ ]:
# Ukažme si, proč je korelace perfektní
print("Teplota roste o:", jednoducha_data['teplota'].diff().dropna().tolist())
print("Zmrzlina roste o:", jednoducha_data['zmrzlina'].diff().dropna().tolist())
print()
print("Každých +5°C → +2 zmrzliny")
print("To je perfektní lineární vztah = korelace 1.0")

### Příklad s menší korelací

In [ ]:
# Realističtější data
realna_data = pd.DataFrame({
    'teplota': [10, 15, 20, 25, 30],
    'zmrzlina': [2, 5, 6, 7, 12]  # ne tak pravidelný růst
})

print("Realističtější data:")
print(realna_data)

pearson_real = realna_data['teplota'].corr(realna_data['zmrzlina'], method='pearson')
print(f"\nPearsonova korelace: {pearson_real:.3f}")
print("Stále silná pozitivní korelace, ale ne perfektní")

## Spearmanův korelační koeficient

**Kdy použít:** Když vás zajímá, zda věci rostou nebo klesají ve stejném pořadí (i když ne stejným tempem).

### Jak Spearmanův koeficient funguje:

- **Převádí hodnoty na pořadí** (ranky): nejmenší = 1, druhý nejmenší = 2, atd.
- **Počítá Pearsonovu korelaci** mezi těmito ranky  
- **Ignoruje konkrétní hodnoty** - zajímá ho pouze pořadí
- **Odolný vůči nelineárním vztahům** - stačí monotónní vztah
- **Méně citlivý na odlehlé hodnoty** než Pearson
- **Vhodný** pro ordinální (pořadová) data

### Krok za krokem s příkladem

In [ ]:
# Data kde Pearson nebude fungovat dobře
nelinearni_data = pd.DataFrame({
    'rychlost': [30, 60, 90, 120, 150],
    'spotreba': [5, 8, 15, 25, 40]  # exponenciální růst
})

print("Data s nelineárním vztahem:")
print(nelinearni_data)
print()

# Porovnání Pearson vs Spearman
pearson = nelinearni_data['rychlost'].corr(nelinearni_data['spotreba'], method='pearson')
spearman = nelinearni_data['rychlost'].corr(nelinearni_data['spotreba'], method='spearman')

print(f"Pearsonova korelace: {pearson:.3f}")
print(f"Spearmanova korelace: {spearman:.3f}")

### Jak Spearman funguje - pořadí

In [ ]:
# Spearman pracuje s pořadím, ne s hodnotami
print("Původní hodnoty:")
print("Rychlost:", nelinearni_data['rychlost'].tolist())
print("Spotřeba:", nelinearni_data['spotreba'].tolist())
print()

print("Pořadí (rank):")
print("Rychlost:", nelinearni_data['rychlost'].rank().tolist())
print("Spotřeba:", nelinearni_data['spotreba'].rank().tolist())
print()

print("Oba rostou ve stejném pořadí → Spearman = 1.0")
print("Ale nerostou lineárně → Pearson < 1.0")

### Praktický příklad - známky vs hodiny studia

In [ ]:
# Známky vs hodiny studia (ne vždy lineární)
studium_data = pd.DataFrame({
    'hodiny_studia': [1, 3, 5, 8, 15],
    'znamka': [5, 4, 3, 2, 1]  # lepší známka = nižší číslo
})

print("Studium vs známky:")
print(studium_data)
print()

pearson_studium = studium_data['hodiny_studia'].corr(studium_data['znamka'], method='pearson')
spearman_studium = studium_data['hodiny_studia'].corr(studium_data['znamka'], method='spearman')

print(f"Pearsonova korelace: {pearson_studium:.3f}")
print(f"Spearmanova korelace: {spearman_studium:.3f}")
print()
print("Více studia → lepší známka (negativní korelace - správně!)")

## Kendallův korelační koeficient (Tau)

**Kdy použít:** Pro malé vzorky nebo když máte odlehlé hodnoty, které by mohly zkreslovat výsledky.

### Jak Kendallův koeficient funguje:

- **Pracuje s párovými porovnáními** - bere každé dva body a porovnává je
- **Concordant páry**: oba body jdou stejným směrem (oba rostou nebo oba klesají)
- **Discordant páry**: body jdou opačným směrem (jeden roste, druhý klesá)
- **Počítá se**: (concordant páry - discordant páry) / (celkový počet párů)
- **Nejodolnější** vůči odlehlým hodnotám
- **Stabilní** i pro malé vzorky (< 30 pozorování)
- **Konzervativní** - obvykle dává nižší hodnoty než Pearson a Spearman

### Krok za krokem s malým vzorkem

In [ ]:
# Malý vzorek dat
maly_vzorek = pd.DataFrame({
    'vek': [20, 25, 30, 35],
    'prijmy': [25000, 35000, 45000, 55000]
})

print("Malý vzorek (4 pozorování):")
print(maly_vzorek)
print()

# Porovnání všech tří metod
pearson_maly = maly_vzorek['vek'].corr(maly_vzorek['prijmy'], method='pearson')
spearman_maly = maly_vzorek['vek'].corr(maly_vzorek['prijmy'], method='spearman') 
kendall_maly = maly_vzorek['vek'].corr(maly_vzorek['prijmy'], method='kendall')

print(f"Pearsonova korelace: {pearson_maly:.3f}")
print(f"Spearmanova korelace: {spearman_maly:.3f}")
print(f"Kendallova korelace: {kendall_maly:.3f}")

### Jak Kendall pracuje s párovými porovnáními

In [ ]:
# Ukážeme si, jak Kendall porovnává páry
jednoduchy_priklad = pd.DataFrame({
    'x': [1, 2, 3, 4],
    'y': [2, 4, 6, 8]
})

print("Jednoduchý příklad pro pochopení Kendall:")
print(jednoduchy_priklad)
print()

# Ručně ukážeme párová porovnání
print("Párová porovnání:")
print("Bod 1 vs Bod 2: x roste (1→2), y roste (2→4) → CONCORDANT")
print("Bod 1 vs Bod 3: x roste (1→3), y roste (2→6) → CONCORDANT") 
print("Bod 1 vs Bod 4: x roste (1→4), y roste (2→8) → CONCORDANT")
print("Bod 2 vs Bod 3: x roste (2→3), y roste (4→6) → CONCORDANT")
print("Bod 2 vs Bod 4: x roste (2→4), y roste (4→8) → CONCORDANT")
print("Bod 3 vs Bod 4: x roste (3→4), y roste (6→8) → CONCORDANT")
print()
print("Celkem 6 párů, všech 6 je concordant, 0 discordant")
print("Kendall = (6-0)/6 = 1.0")

kendall_ukazka = jednoduchy_priklad['x'].corr(jednoduchy_priklad['y'], method='kendall')
print(f"Skutečný výpočet: {kendall_ukazka:.3f}")

### Příklad s discordant páry

In [ ]:
# Ukážeme si data s negativní korelací
negativni_priklad = pd.DataFrame({
    'x': [1, 2, 3, 4],
    'y': [8, 6, 4, 2]  # y klesá když x roste
})

print("Příklad s negativní korelací:")
print(negativni_priklad)
print()

print("Párová porovnání:")
print("Bod 1 vs Bod 2: x roste (1→2), y klesá (8→6) → DISCORDANT")
print("Bod 1 vs Bod 3: x roste (1→3), y klesá (8→4) → DISCORDANT")
print("Bod 1 vs Bod 4: x roste (1→4), y klesá (8→2) → DISCORDANT")
print("(všechny ostatní páry také discordant)")
print()
print("Celkem 6 párů, 0 concordant, 6 discordant")
print("Kendall = (0-6)/6 = -1.0")

kendall_negativni = negativni_priklad['x'].corr(negativni_priklad['y'], method='kendall')
print(f"Skutečný výpočet: {kendall_negativni:.3f}")

### Příklad se smíšenými páry (realističtější data)

In [ ]:
# Realističtější data s některými concordant a discordant páry
smisene_data = pd.DataFrame({
    'x': [1, 2, 3, 4],
    'y': [1, 4, 2, 5]  # ne perfektní růst - někdy klesá
})

print("Realističtější data se smíšenými páry:")
print(smisene_data)
print()

print("Párová porovnání krok za krokem:")
print("Bod 1 vs Bod 2: x roste (1→2), y roste (1→4) → CONCORDANT")
print("Bod 1 vs Bod 3: x roste (1→3), y roste (1→2) → CONCORDANT") 
print("Bod 1 vs Bod 4: x roste (1→4), y roste (1→5) → CONCORDANT")
print("Bod 2 vs Bod 3: x roste (2→3), y klesá (4→2) → DISCORDANT ❌")
print("Bod 2 vs Bod 4: x roste (2→4), y roste (4→5) → CONCORDANT")
print("Bod 3 vs Bod 4: x roste (3→4), y roste (2→5) → CONCORDANT")
print()
print("Celkem: 5 CONCORDANT, 1 DISCORDANT")
print("Kendall = (5-1)/6 = 4/6 = 0.667")

kendall_smiseny = smisene_data['x'].corr(smisene_data['y'], method='kendall')
print(f"Skutečný výpočet: {kendall_smiseny:.3f}")
print()
print("Takto funguje Kendall v praxi - počítá všechny páry jednotlivě!")

### Příklad se stejnými hodnotami (tied páry)

In [ ]:
# Data se stejnými hodnotami
data_se_stejnymi = pd.DataFrame({
    'x': [1, 2, 2, 3],  # dvě stejné hodnoty
    'y': [1, 3, 3, 4]   # dvě stejné hodnoty
})

print("Data se stejnými hodnotami (tied páry):")
print(data_se_stejnymi)
print()

print("Párová porovnání se tied páry:")
print("Bod 1 vs Bod 2: x roste (1→2), y roste (1→3) → CONCORDANT")
print("Bod 1 vs Bod 3: x roste (1→2), y roste (1→3) → CONCORDANT") 
print("Bod 1 vs Bod 4: x roste (1→3), y roste (1→4) → CONCORDANT")
print("Bod 2 vs Bod 3: x stejné (2=2), y stejné (3=3) → TIED ⚬ (ignoruje se)")
print("Bod 2 vs Bod 4: x roste (2→3), y roste (3→4) → CONCORDANT")
print("Bod 3 vs Bod 4: x roste (2→3), y roste (3→4) → CONCORDANT")
print()
print("Celkem: 5 CONCORDANT, 0 DISCORDANT, 1 TIED")
print("Kendall počítá pouze z použitelných párů: (5-0)/5 = 1.0")

kendall_tied = data_se_stejnymi['x'].corr(data_se_stejnymi['y'], method='kendall')
print(f"Skutečný výpočet: {kendall_tied:.3f}")
print()
print("TIED páry se obvykle ignorují nebo mají speciální zacházení")

### Proč je Kendall odolnější vůči odlehlým hodnotám

In [ ]:
# Data s odlehlou hodnotou
s_odlehlou = pd.DataFrame({
    'zkusenosti': [1, 2, 3, 4, 20],  # 20 let = odlehlá hodnota
    'plat': [30000, 35000, 40000, 45000, 48000]  # plat neroste lineárně
})

print("Data s odlehlou hodnotou:")
print(s_odlehlou)
print()

# Porovnání citlivosti metod
pearson_odlehla = s_odlehlou['zkusenosti'].corr(s_odlehlou['plat'], method='pearson')
spearman_odlehla = s_odlehlou['zkusenosti'].corr(s_odlehlou['plat'], method='spearman')
kendall_odlehla = s_odlehlou['zkusenosti'].corr(s_odlehlou['plat'], method='kendall')

print(f"Pearsonova korelace: {pearson_odlehla:.3f}")
print(f"Spearmanova korelace: {spearman_odlehla:.3f}")
print(f"Kendallova korelace: {kendall_odlehla:.3f}")
print()
print("Proč je Kendall nejstabilnější:")
print("- Pearson: ovlivněn extrémní hodnotou 20 let")
print("- Spearman: pracuje s ranky, ale stále citlivý na pořadí")  
print("- Kendall: každý pár má stejnou váhu, extrém má menší vliv")

## Kdy použít který koeficient - praktický průvodce

### Rozhodovací tabulka
| Situace | Doporučený koeficient | Jak funguje |
|---------|----------------------|-------------|
| Lineární vztah, velký vzorek | **Pearson** | Kovariance / směr. odchylky |
| Nelineární, ale monotónní vztah | **Spearman** | Pearson na ranky |
| Malý vzorek (< 30) | **Kendall** | Párová porovnání |
| Odlehlé hodnoty | **Kendall** | Méně citlivý na extrémy |
| Pořadová data (1., 2., 3.) | **Spearman** nebo **Kendall** | Navržené pro pořadí |

### Praktické srovnání na jednom datasetu

In [ ]:
# Komplexní příklad s různými vztahy
srovnani_data = pd.DataFrame({
    'linearni': [1, 2, 3, 4, 5],           # perfektní lineární
    'kvadraticka': [1, 4, 9, 16, 25],      # nelineární, ale monotónní  
    's_sumem': [1, 2, 3, 4, 100],          # s odlehlou hodnotou
    'nahodna': [3, 1, 4, 2, 5]             # žádný vztah
})

print("Testovací data:")
print(srovnani_data)
print()

# Srovnání všech metod pro každý pár
promenne = ['linearni', 'kvadraticka', 's_sumem', 'nahodna']

print("Korelace s 'linearni' proměnnou:")
print("=" * 50)

for promenna in promenne:
    if promenna != 'linearni':
        p = srovnani_data['linearni'].corr(srovnani_data[promenna], method='pearson')
        s = srovnani_data['linearni'].corr(srovnani_data[promenna], method='spearman')
        k = srovnani_data['linearni'].corr(srovnani_data[promenna], method='kendall')
        
        print(f"\n{promenna}:")
        print(f"  Pearson:  {p:.3f}")
        print(f"  Spearman: {s:.3f}")
        print(f"  Kendall:  {k:.3f}")

## Praktické tipy pro začátečníky

### 1. Začněte vždy s Pearsonem

In [ ]:
# Základní kontrola
jednoduche_data = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [2, 4, 6, 8, 10]
})

# Začněte s Pearsonem
korelace = jednoduche_data.corr(method='pearson')
print("Začněte vždy s Pearsonem:")
print(korelace)

### 2. Vizuálně zkontrolujte data

In [ ]:
# Jednoduchá kontrola vztahu
print("Kontrola vztahu očima:")
print("X roste:", jednoduche_data['x'].is_monotonic_increasing)
print("Y roste:", jednoduche_data['y'].is_monotonic_increasing)
print("Oba rostou → pozitivní korelace")

### 3. Testujte všechny tři, když si nejste jisti

In [ ]:
def porovnej_korelace(data, sloupec1, sloupec2):
    """Porovná všechny tři typy korelace"""
    p = data[sloupec1].corr(data[sloupec2], method='pearson')
    s = data[sloupec1].corr(data[sloupec2], method='spearman')
    k = data[sloupec1].corr(data[sloupec2], method='kendall')
    
    print(f"Korelace {sloupec1} vs {sloupec2}:")
    print(f"  Pearson:  {p:.3f}")
    print(f"  Spearman: {s:.3f}")
    print(f"  Kendall:  {k:.3f}")
    
    # Jednoduchá interpretace
    if abs(p - s) < 0.1:
        print("  → Vztah je pravděpodobně lineární")
    else:
        print("  → Vztah může být nelineární")

# Test funkce
test_data = pd.DataFrame({
    'a': [1, 2, 4, 8, 16],
    'b': [1, 2, 3, 4, 5]
})

porovnej_korelace(test_data, 'a', 'b')

## Časté chyby začátečníků

### Chyba 1: Používání pouze Pearsona

In [ ]:
# Špatně - pouze Pearson pro nelineární data
nelinearni = pd.DataFrame({
    'x': [1, 2, 3, 4, 5],
    'y': [1, 4, 9, 16, 25]  # x na druhou
})

print("Pouze Pearson (nesprávně):")
print(f"Korelace: {nelinearni['x'].corr(nelinearni['y']):.3f}")
print()

print("Správně - porovnání:")
print(f"Pearson:  {nelinearni['x'].corr(nelinearni['y'], method='pearson'):.3f}")
print(f"Spearman: {nelinearni['x'].corr(nelinearni['y'], method='spearman'):.3f}")

### Chyba 2: Nerozpoznání odlehlých hodnot

In [ ]:
# Data s extrémní hodnotou
extremni_data = pd.DataFrame({
    'normalni': [1, 2, 3, 4, 5],
    'extremni': [1, 2, 3, 4, 1000]  # 1000 je extrém
})

print("S extrémní hodnotou:")
print(f"Pearson (citlivý):   {extremni_data['normalni'].corr(extremni_data['extremni'], method='pearson'):.3f}")
print(f"Kendall (odolný):    {extremni_data['normalni'].corr(extremni_data['extremni'], method='kendall'):.3f}")

## Shrnutí pro začátečníky

### Jednoduchý algoritmus rozhodování:

1. **Máte lineární vztah a spojitá data?** → Použijte **Pearson**
   - Počítá z původních hodnot
   - Nejpřesnější pro lineární vztahy
   
2. **Vztah není lineární, ale jde "nahoru/dolů"?** → Použijte **Spearman**  
   - Pracuje s pořadím hodnot
   - Odolný vůči nelineárním vztahům
   
3. **Máte málo dat (< 30) nebo divné hodnoty?** → Použijte **Kendall**
   - Porovnává páry bodů
   - Nejstabilnější a nejodolnější
   
4. **Nejste si jisti?** → Vypočítejte všechny tři a porovnejte

### Základní interpretace:
- **0.0-0.3**: Slabý vztah
- **0.3-0.7**: Střední vztah  
- **0.7-1.0**: Silný vztah
- **Záporné hodnoty**: Opačný vztah (jedno roste, druhé klesá)

**Pamatujte**: Korelace neznamená, že jedno způsobuje druhé!